In [1]:
import cdsapi
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.prepared import prep
import geopandas
from datetime import datetime, timedelta
import os

In [ ]:
#get the data 
dataset = "reanalysis-era5-single-levels"
request = {"product_type": ["reanalysis"], "variable": ["2m_dewpoint_temperature", "2m_temperature"], "year": ["2024", "2025"], "month": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"], "day": ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"], "time": ["00:00", "01:00", "02:00", "03:00", "04:00", "05:00", "06:00", "07:00", "08:00", "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"], "data_format": "netcdf", "download_format": "zip", "area": [51, -5, 41.5, 9.75]}
client = cdsapi.Client()
client.retrieve(dataset, request, "2024-2025.zip")

In [5]:
files = [file for file in os.listdir("data/ERA5") if file[-3:] == ".nc"]
files = sorted(files)
files

['1940-1945.nc',
 '1946-1951.nc',
 '1952-1957.nc',
 '1958-1963.nc',
 '1964-1969.nc',
 '1970-1975.nc',
 '1976-1981.nc',
 '1982-1987.nc',
 '1988-1993.nc',
 '1994-1999.nc',
 '2000-2005.nc',
 '2006-2011.nc',
 '2012-2017.nc',
 '2018-2023.nc',
 '2024-2025.nc']

In [3]:
#for data of shape (n, x, y) with n the hour, x the latitude and y the longitude 
#we can find daily datas in the form (0:23, x, y), (24:47, x, y), (48:71, x, y)....
#then make the calcalutions on the daily data, find the data points of each dep, find the day and finaly save the produces datas

In [6]:
geo = geopandas.read_file("data/geojsonfrance_corse_20.json") #read france departement geometries
geo["code"] = geo["code"].astype(int)
geo = geo.sort_values(by="code").reset_index(drop=True)

In [7]:
points = []
data_index = 0
points = dict()
latitude = Dataset("data/ERA5/1940-1945.nc").variables["latitude"][:]
longitude = Dataset("data/ERA5/1940-1945.nc").variables["longitude"][:]
#for every lat and lon, we make a dict of index POINT(lon, lat) and value the index of the data associated with this point
for lat in latitude:
    for lon in longitude:
        points[Point(lon, lat)] = data_index
        data_index +=1

In [ ]:
result = []
for file in files[13:]:
    print(file) #keep track of script
    temp = Dataset(f"data/ERA5/{file}").variables["t2m"][:] - 273.15 #convert to from Kelvin to Celsius
    dewP = Dataset(f"data/ERA5/{file}").variables["d2m"][:] - 273.15 #convert to from Kelvin to Celsius
    rh = 100*(np.exp((17.625*dewP)/(243.04+dewP))/np.exp((17.625*temp)/(243.04+temp))) #August-Roche-Magnus equation to calculate relative humidity
    vpd = 6.1078 * (1 - rh / 100) * np.exp(17.08085 * temp / (234.175 + temp)) * 0.1 #vpd in kPa #Alduchov, O. A., and R. E. Eskridge, 1996 formula
    vpd = vpd.filled(np.nan) #replace masked values by NaN
    
    startYear = int(file[:4]) #get start year of each file
    endYear = int(file[5:9]) #get end year of each file
    date = datetime(startYear, 1, 1) #make a new datetime for the first day of the file
    
    #calculate how many complete days we have
    total_hours = vpd.shape[0]
    hours_per_day = 24
    num_complete_days = total_hours // hours_per_day
    
    #loop through the array in 24-hour chunks
    for day in range(num_complete_days):
        #calculate start and end indices for this day
        start_idx = day * hours_per_day       # 0, 24, 48, ...
        end_idx = start_idx + hours_per_day   # 24, 48, 72, ...
        
        # Get the data for this day
        dailyVpd = vpd[start_idx:end_idx]
    
        vpd_max = np.max(dailyVpd, axis=0)
        vpd_max_flattened = vpd_max.flatten()
        vpd_min = np.min(dailyVpd, axis=0)
        vpd_min_flattened = vpd_min.flatten()
        vpd_mean = np.mean(dailyVpd, axis=0)
        vpd_mean_flattened = vpd_mean.flatten()
    
        for _, dep in geo.iterrows():
            prepared = prep(dep["geometry"]) #use prep for batch operations
            valid_points = []
            valid_points.extend(filter(prepared.contains, points)) #find POINTS in dep
            valid_indices = [points[point] for point in valid_points if point in points] #make a list of valid points that are in the dep

            if valid_indices:
                vpd_max_dep = np.mean(vpd_max_flattened[valid_indices]) #find values with indices
                vpd_mean_dep = np.mean(vpd_mean_flattened[valid_indices]) #find values with indices
                vpd_min_dep = np.mean(vpd_min_flattened[valid_indices]) #find values with indices
            else:
                vpd_max_dep = np.nan
                vpd_mean_dep = np.nan
                vpd_min_dep = np.nan
            
            result.append({"date": date, "departement": dep["nom"], "dep": dep["code"], "vpd_max": float(vpd_max_dep), "vpd_mean": float(vpd_mean_dep), "vpd_min": float(vpd_min_dep)})
        
        date += timedelta(days=1) #append one day to datetime
    df = pd.DataFrame(result)
    df.to_csv(f"data/ERA5/1940-{endYear}_vpd.csv")        

2018-2023.nc
